# Toronto neighborhoods geocoordenates

### 1. Creating the dataframe using BeautifulSoup (skip to 2 to see the final dataframe)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [ ]:
# create html variable and an empty dataframe
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_hoods = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

In [ ]:
# open html using BeautifulSoup
source = requests.get(url).text
soup = BeautifulSoup(source, 'html.parser')

In [ ]:
# table separeted from the webpage using soup atributes
table = soup.table
table_rows = table.tbody.text

# create a list with each row of table as a list element
toronto_list = table_rows.split('\n\n\n')

# drop of the table head - the first element of the list
toronto_list.pop(0)
print(toronto_list)

In [ ]:
# separate each postalcode, borough, neighborhood from the list to a separate one

postalcode = []
borough = []
neighborhood = []
    
for i in toronto_list:
    row = i
    row = row.split('\n')
    if row[1] != 'Not assigned':
        postalcode.append(row[0])
        borough.append(row[1])
        neighborhood.append(row[2]) 
        
    else:
        pass

In [ ]:
#alocate each lists values into the dataframe

for i in range(len(postalcode)):
    toronto_hoods = toronto_hoods.append({'PostalCode' : postalcode[i] , 'Borough' : borough[i], 'Neighborhood' : neighborhood[i]}, ignore_index=True)

toronto_hoods.head(10)

In [ ]:
# check cases of missing neighborhoods
toronto_hoods.replace('Not assigned', np.nan, inplace=True)
toronto_hoods.isnull().sum()

In [ ]:
# replace missing neighborhoods
toronto_hoods.loc[6].replace(np.nan, 'Queen\'s Park', inplace=True)

In [ ]:
print(type(toronto_hoods))

In [ ]:
# aggregate neighborhoods according to postalcode
strJoin = lambda x:", ".join(x.astype(str))
toronto_hoods = toronto_hoods.groupby(['PostalCode', 'Borough'], as_index=False, sort=False).agg({'Neighborhood':strJoin})

In [ ]:
toronto_hoods.head()

In [ ]:
toronto_hoods.shape

### 2. Getting geocoordenates and creating final dataframe

In [ ]:
coordenates = 'https://cocl.us/Geospatial_data'
latlong = pd.read_csv(coordenates)
latlong.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
latlong.head()

In [ ]:
latlong.shape

In [ ]:
toronto_final = pd.merge(toronto_hoods, latlong, sort=True)

In [ ]:
toronto_final.head()

In [ ]:
toronto_final.to_csv('toronto_coursera', index = False)

In [ ]:
toronto_final.shape